# How to do batch hyperparmaeter tuning with sci-kit using quante_carlo

This is a demonstration that trains sci-kit models on toy datasets and makes reuqests to bayesian optimization as a service for hyperparameter tuning.<br>
Note to person giving demo:<br>
    <code>gunicorn -w 18 'flask_worker:app'</code><br>
(if you hvae 18 processors to run the bo)

<code>pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12==24.10.* cuml-cu12==24.10.* </cide>

For this demo, there is no actual parallel training. The purpose of this demo is so that you can see how the parallel HPT works even if you don't have the ability to do training in parallel.

## Load Libraries

### Python

In [ ]:
import requests 
import json
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes, load_digits, load_breast_cancer
from sklearn.model_selection import train_test_split
diabetes = load_diabetes()
digits = load_diabetes()
breast_cancer = load_breast_cancer()

from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import time
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder

import cupy as cp
from cuml.ensemble import RandomForestRegressor as curfr

import pandas as pd
from sklearn.cluster import KMeans
import os
from sklearn.tree import DecisionTreeRegressor

### User defined modules

#### Random forest implementation using NVIDIA's rapids library
Runs RF in batch mode due to very large data frame size

In [ ]:
class batch_rf:
    def __init__(self, max_depth, min_samples_leaf, min_samples_split, impurity_decrease):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.impurity_decrease = impurity_decrease
        self.all_predictions = []
        self.truth = []

    def get_file(self, path, f):
        if f == 'xaa':
            nvidia_df = pd.read_csv(path + '/' + f, nrows=100)
        else:
            headers = pd.read_csv(path + '/xaa', nrows=1)
            nvidia_df = pd.read_csv(path + '/' + f, nrows=100, header=None)
            nvidia_df.columns = headers.columns
            
        float_cols = [c for c in nvidia_df if nvidia_df[c].dtype == "float64"]
        float32_cols = {c: np.float32 for c in float_cols}
        if f == 'xaa':
            nvidia_df = pd.read_csv(path + '/' + f, dtype=float32_cols)
        else:
            nvidia_df = pd.read_csv(path + '/' + f, dtype=float32_cols, header=None)
            nvidia_df.columns = headers.columns
        return nvidia_df
        
    def prep(self, df, enc=None, kmodel=None):
            
        continuous_predictors = df.columns[4:]
        categorical = ['trickortreat', 'kingofhalloween']
        if not enc:
            enc = OneHotEncoder(handle_unknown='ignore')
            enc = enc.fit(df[categorical])
            
        categorical_df = enc.transform(df[categorical]).astype('float32')
    
        if not kmodel:
            k = KMeans(n_clusters=20)
            k.fit(categorical_df)
            leaf_ids = k.predict(categorical_df)
        else:
            k = kmodel
            leaf_ids = kmodel.predict(categorical_df)
            
        df2 = df[continuous_predictors].copy()
        df2['dtree_id'] = leaf_ids.astype('float32')
        df2 = df2.copy().astype('float32')
        
        # impute missing
        for c in df2.columns:
            m = df2[c].mean()
            df2[c] = df2[c].fillna(value=m)
                
        return df2, enc, k
        

    def fit(self, path):
                
        files = os.listdir(path)
        for f in files[:3]:
            if f[0] == 'x':
                df = self.get_file(path, f)
                df2, enc, kmodel = self.prep(df)
                
                X_train, X_test, y_train, y_test = train_test_split(df2, df['y'], test_size=0.3, random_state=42)
            
                rf = curfr(max_depth=self.max_depth, 
                           min_samples_leaf=self.min_samples_leaf,
                           min_samples_split=self.min_samples_split,
                           min_impurity_decrease=self.impurity_decrease,
                           n_estimators=200, accuracy_metric='rmse')

                rf.fit(X_train, y_train)
                
                predictions = []        
                for g in files[:3]:
                    if g[0] == 'x':
                        if g[0] != f[0]:
                            df2 = self.get_file(path, g)
                            df3 = self.prep(df2, enc, kmodel)
                            X_train2, X_test2, y_train2, y_test2 = train_test_split(df3, df2['y'], test_size=0.3, random_state=42)
                            predictions.append(rf.predict(X_test2))
                        else:
                            predictions.append(rf.predict(X_test))
                            self.truth.append(y_test.copy())

                self.all_predictions.append(predictions.copy())
                
        return self
        
    def score(self):
        sq_error = 0
        sum_of_squares = 0
        m = 0
        s = 0
        for t in self.truth:
            m += np.sum(t)
            s += len(t)
            
        for a in range(len(self.all_predictions)):
            sq_error += np.sum([(p-t)**2 for p, t in zip(self.all_predictions[a], self.truth[a])])
            sum_of_squares += np.sum([(p-m/s)**2 for p in self.all_predictions[a]])
        return 2-sq_error/sum_of_squares


#### Wrapper for multiple different models

In [ ]:
class bayes_optimization_example:
    def __init__(self, gbr_batch_size, n_processors, model):
        self.n_procs = n_processors
        self.gbr_batch_size = gbr_batch_size # how many points to evaluate when optimizing gaussian process
        self.model = model

    def metric(self, x, y):
#        if self.target_type == 'regression':
#            if x < y:
#                return True
#            else:
#                return False
#        else:
        if x > y:
            return True
        else:
            return False
            
    def model_selection(self, r):
        if self.model == 'ElasticNet':
            self.target_type = 'regression'
            model = ElasticNet(alpha = r[0], l1_ratio=r[1])
        elif self.model == 'SVR':
            self.target_type = 'regression'
            model = make_pipeline(StandardScaler(), SVR(C=r[0], epsilon=r[1]))
        elif self.model == 'XGBoostClassifier':
            self.target_type = 'classification'
            model = XGBClassifier(gamma=r[0], reg_lambda=r[1], colsample_bytree=r[2], 
                                  max_depth=r[3], min_child_weight=r[4], learning_rate=r[5])
        elif self.model == 'XGBoostRegressor':
            self.target_type = 'regression'
            model = XGBRegressor(gamma=r[0], reg_lambda=r[1], colsample_bytree=r[2], 
                                 max_depth=r[3], min_child_weight=r[4], learning_rate=r[5])
        elif self.model == 'RandomForestRegressor':
            self.target_type = 'regression'
            model = batch_rf(r[0], r[1], r[2], r[3])
#            model = curfr(max_depth=r[0], n_bins=r[2],
#            model = curfr(max_depth=r[0], max_features=1.0, n_bins=512,
#                          min_samples_leaf=r[3],
#                          min_samples_split=r[4],
#                          min_impurity_decrease=r[5],
#                          n_estimators=200, accuracy_metric='r2')
        else:
            print('No Model')
        return model
    
    def initialize(self, toy_data, hp_types, hp_ranges):
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(toy_data.data, toy_data.target, test_size=0.3, random_state=42)
        self.data = toy_data.data
        self.target = toy_data.target
        self.hp_types = hp_types
        self.historical_scores = []
        self.historical_points = []
        self.hp_ranges = hp_ranges
        for a in range(3):
            r = [random.uniform(hp_ranges[x][0], hp_ranges[x][1]) 
                 if hp_types[x] == 'float' else random.randint(hp_ranges[x][0], hp_ranges[x][1]) 
                 for x in range(len(hp_ranges))]

            model = self.model_selection(r)

            if self.model != 'RandomForestRegressor':
                model.fit(self.X_train, self.y_train)
                if self.target_type == 'regression':
                    self.historical_scores.append(str(r2_score(self.y_test, model.predict(self.X_test))))
                else:
                    self.historical_scores.append(str(cross_val_score(model, toy_data.data, toy_data.target).mean()))                
            else:
                fitted_model = model.fit('split_training_data')
                self.historical_scores.append(fitted_model.score())

                
            self.historical_points.append(','.join([str(x) for x in r]))

    def test_points(self, next_points):
        
        for nxt_pt in [p.split(',') for p in next_points['next_points'].split(';')]:
            r = [float(nxt_pt[x]) if self.hp_types[x] == 'float' else int(nxt_pt[x]) for x in range(len(nxt_pt))]
            
            model = self.model_selection(r)
            
            if self.model != 'RandomForestRegressor':
                model.fit(self.X_train, self.y_train)
                if self.target_type == 'regression':
                    self.historical_scores.append(str(r2_score(self.y_test, model.predict(self.X_test))))
                else:
                    self.historical_scores.append(str(cross_val_score(model, self.data, self.target).mean()))               
            else:
                fitted_model = model.fit('split_training_data')
                self.historical_scores.append(fitted_model.score())
                
        self.historical_points += next_points['next_points'].split(';')
        
    def get_best_point(self):
        
        #if self.target_type == 'regression':
        #    best = 1000
        #else:
        best = -100
            
        best_point = 'failed'
        for s, pt in zip(qc_bo.historical_scores, qc_bo.historical_points):
            if self.metric(float(s),  best):
                best = float(s)
                best_point = pt
        return(best_point)
    
    def create_url (self):

#        if self.target_type == 'regression':
#            # if lower score is better (rmse, etc)
#            h_scores = [float(s) for s in self.historical_scores]
#            mx = np.max(h_scores)
#            data = json.dumps({'scores': ','.join([str(1+mx-s) for s in h_scores]), 'points': ';'.join(self.historical_points)})
#        else:
        data = json.dumps({'scores': ','.join([str(s) for s in self.historical_scores]), 'points': ';'.join(self.historical_points)})
        
        y_best = 10
        hp_ranges_str = ';'.join([','.join([str(x) for x in s]) for s in self.hp_ranges])
        hp_types_str = ','.join(self.hp_types)
        stem = "http://localhost:8000/bayes_opt?hp_types="
        url = stem + "{}&g_batch_size={}&hp_ranges={}&y_best={}&n_gpus={}&use_qc={}".format(hp_types_str, self.gbr_batch_size, 
                                                                                               hp_ranges_str, y_best, self.n_procs, 'False')
        return url, data

In [ ]:
class use_log:
    def __init__(self, log_or_not):
        self.log_or_not = log_or_not
    def log(self, x):
        if self.log_or_not:
            return np.log(x)
        else:
            return (x)
def show_results(session, log):
    u = use_log(log)
    h = [u.log(float(q)) for q in session.historical_scores]
    print("average performance (during bo)               {}".format(np.mean([float(q) for q in session.historical_scores])))
    print("standard deviation of performance (during bo) {}".format(np.std([float(q) for q in session.historical_scores])))
#    if session.target_type == 'regression':
#        best = 10
#    else:
    best = -100
    
    best_so_far = []
    for q in session.historical_scores:
        if session.metric(u.log(float(q)), best):
            best = u.log(float(q))
        best_so_far.append(best)
    plt.plot(h, label='historical')
    plt.plot(best_so_far, label='best_so_far')
    p = plt.legend()
    print("Best after BO {}".format(best))

class bunch:
    def __init__(self, d):
        self.data = d['data']
        self.target = d['target']

## Elastic Net

In [ ]:
qc_bo = bayes_optimization_example(30, 4, 'EN')
hp_types = ['float', 'float']
hp_ranges =  [[0.0001,.99999],[0.0001,.99999]]
qc_bo.initialize(diabetes, hp_types, hp_ranges)
historical_qei = []
best_points = []

In [ ]:
for a in range(50):
    url, data = qc_bo.create_url() # using historical data
    response = requests.post(url, data=data)
    next_points = json.loads(response.text)
    historical_qei.append(next_points['best_ccdf'])
    qc_bo.test_points(next_points)
    best_points.append(qc_bo.get_best_point())

In [ ]:
h = [np.log(float(q)) for q in qc_bo.historical_scores]
best = 10
best_so_far = []
for q in qc_bo.historical_scores:
    if np.log(float(q))< best:
        best = np.log(float(q))
    best_so_far.append(best)
plt.plot(h)
plt.plot(best_so_far)

In [ ]:
# when parameters changed
plt.plot([float(b.split(',')[0]) for b in best_points], label='alpha')
plt.plot([float(b.split(',')[1]) for b in best_points], label='l1_ratio')
p = plt.legend()
qc_bo.get_best_point()

In [ ]:
plt.plot(historical_qei)

## Support Vector Regression
C, 
epsilon

In [ ]:
qc_bo = bayes_optimization_example(30, 4, 'SVR')
hp_types = ['float', 'float']
hp_ranges =  [[0.1,10],[0.001,.999]]
qc_bo.initialize(diabetes, hp_types, hp_ranges)
historical_qei = []
best_points = []

In [ ]:
for a in range(50):
    url, data = qc_bo.create_url() # using historical data
    response = requests.post(url, data=data)
    next_points = json.loads(response.text)
    historical_qei.append(next_points['best_ccdf'])
    qc_bo.test_points(next_points)
    best_points.append(qc_bo.get_best_point())

In [ ]:
# when parameters changed
plt.plot([float(b.split(',')[0]) for b in best_points], label='C')
plt.plot([float(b.split(',')[1]) for b in best_points], label='epsilon')
p = plt.legend()
qc_bo.get_best_point()

In [ ]:
h = [np.log(float(q)) for q in qc_bo.historical_scores]
best = 10
best_so_far = []
for q in qc_bo.historical_scores:
    if np.log(float(q))< best:
        best = np.log(float(q))
    best_so_far.append(best)
plt.plot(h)
plt.plot(best_so_far)

## XGBoost Classifier

### One hot encoding for digits dataset

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(digits.target.reshape(-1,1))
enc.categories_
ohe_target = enc.transform(digits.target.reshape(-1, 1))
#enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

In [ ]:
digits_bunch= bunch({'target': ohe_target,
                     'data': digits.data})

### Breast Cancer

In [ ]:
qc_bo = bayes_optimization_example(300, 4, 'XGBoost')

parameter_names = ['gamma', 'reg_lambda', 'colsample_by_tree',
                   'max_depth', 'min_child_weight', 'learning_rate']

hp_types = ['float', 'float', 'float',  'int', 'float', 'float']
hp_ranges =  [[0.01, .999],[0.001,.999], [0.001,.999],
              [2, 5],[0.001,.999] ,[0.001,.999]]

qc_bo.initialize(breast_cancer, hp_types, hp_ranges)
historical_qei = []
best_points = []

In [ ]:
for a in range(100):
    url, data = qc_bo.create_url() # using historical data
    start = time.time()
    response = requests.post(url, data=data)
    print("{}: Spent {} seconds getting next points".format(a, round(time.time()-start,3)))
    next_points = json.loads(response.text)
    historical_qei.append(next_points['best_ccdf'])
    qc_bo.test_points(next_points)
    best_points.append(qc_bo.get_best_point())

In [ ]:

show_results(qc_bo, False)

In [ ]:
[str(p)+': '+z for p, z in zip(parameter_names, qc_bo.get_best_point().split(','))]

## XGBoost Regressor

In [ ]:
qc_bo = bayes_optimization_example(300, 4, 'XGBoostRegressor')

parameter_names = ['gamma', 'reg_lambda', 'colsample_by_tree',
                   'max_depth', 'min_child_weight', 'learning_rate']

hp_types = ['float', 'float', 'float',  'int', 'float', 'float']
hp_ranges =  [[0.01, .999],[0.001,.999], [0.001,.999],
              [2, 5],[0.001,.999] ,[0.001,.999]]

qc_bo.initialize(diabetes, hp_types, hp_ranges)
historical_qei = []
best_points = []

In [ ]:
for a in range(10):
    url, data = qc_bo.create_url() # using historical data
    start = time.time()
    response = requests.post(url, data=data)
    print("{}: Spent {} seconds getting next points".format(a, round(time.time()-start,3)))
    next_points = json.loads(response.text)
    historical_qei.append(next_points['best_ccdf'])
    qc_bo.test_points(next_points)
    best_points.append(qc_bo.get_best_point())

In [ ]:
show_results(qc_bo, False)

## NVIDIA Hackathon

In [ ]:
qc_bo = bayes_optimization_example(480, 2, 'RandomForestRegressor')
# can't tweak max_samples and n_bins together
# n_bins can't be too large with large n clusters

parameter_names = ['max_depth',  'min_samples_leaf', 
                   'min_samples_split', 'max_samples', 'min_impurity_decrease'] 
hp_types = ['int', 'int', 'int', 'int', 'float']
hp_ranges =  [[3, 30], [2, 20], [2, 20], [10, 256], [.1,.99]]

qc_bo.initialize(breast_cancer, hp_types, hp_ranges)
historical_qei = []
best_points = []

# to do:
# flask_worker validate input

In [ ]:
for a in range(100):
    url, data = qc_bo.create_url() # using historical data
    start = time.time()
    response = requests.post(url, data=data)
    print("{}: Spent {} seconds getting next points".format(a, round(time.time()-start,3)))
    next_points = json.loads(response.text)
    historical_qei.append(next_points['best_ccdf'])
    qc_bo.test_points(next_points)
    best_points.append(qc_bo.get_best_point())

In [ ]:
show_results(qc_bo, False)

In [ ]:
[str(p)+': '+z for p, z in zip(parameter_names, get_best_point(qc_bo).split(','))]